# Install all the necessary libraries for running the scripts

In [4]:
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Adding Counties to the Addresses
Zip code to County crosswalk file provided by the U.S. Office of Policy Development and Research (PD&R) is used to add county names to the addresses. The crosswalk file is available at https://www.huduser.gov/apps/public/uspscrosswalk/home

In [11]:
def get_county_from_address(address):
    """Fetch county from address using Nominatim API."""
    geolocator = Nominatim(user_agent="YourApp_NameHere")  # Change 'YourApp_NameHere' to something descriptive
    try:
        location = geolocator.geocode(address, timeout=10)
        
        if location:
            # Split the address components
            address_components = location.address.split(", ")
            for component in address_components:
                if "County" in component:
                    sleep(1)  # Introduce delay after successful request
                    return component
        return None
    except Exception as e:
        print(f"Error geocoding address: {address}. Error: {e}")
        return None

In [12]:
def geocode_dataframe(df):
    """Geocode FULL_ADDRESS column and add COUNTY column."""
    # Apply geocoding function to get county
    df['COUNTY'] = df['FULL_ADDRESS'].apply(get_county_from_address)
    # Return the updated dataframe
    return df

In [13]:
df_with_county = geocode_dataframe(df)

Error geocoding address: 515 FOUNTAIN AVENUE, BROOKLYN, NY 11208. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 1150 UNION AVENUE, BRONX, NY 10459. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 818 HOME STREET, BRONX, NY 10459. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 201 WEST 93RD STREET, NEW YORK, NY 10025. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 868 AMSTERDAM AVENUE, NEW YORK, NY 10025. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 825 COLUMBUS AVENUE, NEW YORK, NY 10025. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 55 WEST 100TH STREET, NEW YORK, NY 10025. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 845 COLUMBUS AVENUE, NEW YORK, NY 10025. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 865 COLUMBUS AVENUE, NEW YORK, NY 10025. Error: [Errno 11001] getaddrinfo failed
Error geocoding address: 60 WEST 104TH STREET, NEW YORK

In [ ]:
df_with_county.to_csv('updated_dataframe.csv', index=False)

# Match the county and zip to the fips code

# Finally merge the two dataframes